### 1: Setup and Load Data

In [4]:
import pandas as pd

# Load the two generated datasets
try:
    subscriber_df = pd.read_csv('monthly_subscriber_data.csv')
    population_df = pd.read_csv('kebele_population.csv')
    
    print("Data loading successful.")
    print(f"Subscriber data rows: {len(subscriber_df)}")
    print(f"Population data rows: {len(population_df)}")
    
except FileNotFoundError:
    print("ERROR: One or both required CSV files were not found. Please run the data generation script first.")

Data loading successful.
Subscriber data rows: 1080
Population data rows: 15


#### 2: Initial Data Cleaning and Preparation

In [6]:
# Convert 'Date' column to datetime objects
subscriber_df['Date'] = pd.to_datetime(subscriber_df['Date'])

# Ensure data is sorted by Kebele, Segment, and Date for correct cumulative calculations
subscriber_df = subscriber_df.sort_values(by=['Kebele', 'Segment', 'Date']).reset_index(drop=True)

# Calculate Net Acquisition per segment per month
subscriber_df['Net_Acquisition'] = subscriber_df['New_Subscribers'] - subscriber_df['Monthly_Churn']

print("Date conversion, sorting, and Net Acquisition calculation complete.")
print(subscriber_df[['Date', 'Kebele', 'Segment', 'Net_Acquisition']].head().to_markdown(index=False))

Date conversion, sorting, and Net Acquisition calculation complete.
| Date                | Kebele   | Segment   |   Net_Acquisition |
|:--------------------|:---------|:----------|------------------:|
| 2023-01-01 00:00:00 | Arada    | Business  |                 4 |
| 2023-02-01 00:00:00 | Arada    | Business  |                 5 |
| 2023-03-01 00:00:00 | Arada    | Business  |                 9 |
| 2023-04-01 00:00:00 | Arada    | Business  |                 5 |
| 2023-05-01 00:00:00 | Arada    | Business  |                 8 |


#### 3: Calculate Segment-Wise Cumulative Subscribers

In [7]:
# Group by Kebele and Segment, then calculate the running sum (cumsum) of Net_Acquisition
subscriber_df['Cumulative_Subscribers'] = subscriber_df.groupby(['Kebele', 'Segment'])['Net_Acquisition'].cumsum()

print("Segment-wise cumulative subscriber count calculated.")
print(subscriber_df.tail(3).to_markdown(index=False))

Segment-wise cumulative subscriber count calculated.
| Date                | Kebele    | Segment          |   New_Subscribers |   Monthly_Churn |   Net_Acquisition |   Cumulative_Subscribers |
|:--------------------|:----------|:-----------------|------------------:|----------------:|------------------:|-------------------------:|
| 2024-10-01 00:00:00 | Wadu Amba | Telebirr Adopter |                38 |               6 |                32 |                      517 |
| 2024-11-01 00:00:00 | Wadu Amba | Telebirr Adopter |                34 |               1 |                33 |                      550 |
| 2024-12-01 00:00:00 | Wadu Amba | Telebirr Adopter |                25 |               5 |                20 |                      570 |


#### 4: Calculate Kebele-Total Cumulative Subscribers

In [8]:
# Calculate the total net acquisition across all segments for each Kebele and Date
kebele_totals = subscriber_df.groupby(['Kebele', 'Date'])[['Net_Acquisition']].sum().reset_index()
kebele_totals.rename(columns={'Net_Acquisition': 'Kebele_Net_Acquisition'}, inplace=True)

# Calculate the running total for the entire kebele
kebele_totals['Kebele_Cumulative_Subscribers'] = kebele_totals.groupby('Kebele')['Kebele_Net_Acquisition'].cumsum()

# Merge this total cumulative count back into the main DataFrame
subscriber_df = pd.merge(
    subscriber_df, 
    kebele_totals[['Kebele', 'Date', 'Kebele_Cumulative_Subscribers']], 
    on=['Kebele', 'Date'], 
    how='left'
)

print("Kebele-total cumulative subscriber count calculated and merged.")
print(subscriber_df[['Date', 'Kebele', 'Kebele_Cumulative_Subscribers']].tail(3).to_markdown(index=False))

Kebele-total cumulative subscriber count calculated and merged.
| Date                | Kebele    |   Kebele_Cumulative_Subscribers |
|:--------------------|:----------|--------------------------------:|
| 2024-10-01 00:00:00 | Wadu Amba |                            1955 |
| 2024-11-01 00:00:00 | Wadu Amba |                            2055 |
| 2024-12-01 00:00:00 | Wadu Amba |                            2132 |


#### 5: Merge Population Data and Calculate Market Penetration Rate

In [9]:
# Merge subscriber data with the static population data
analysis_df = pd.merge(
    subscriber_df, 
    population_df[['Kebele', 'Population', 'Type']], 
    on='Kebele', 
    how='left'
)

# Calculate the Market Penetration Rate (per kebele)
# Formula: (Kebele Cumulative Subscribers / Kebele Population) * 100
analysis_df['Market_Penetration_Rate'] = (
    analysis_df['Kebele_Cumulative_Subscribers'] / analysis_df['Population']
) * 100

print("Market Penetration Rate calculated and population data merged.")
print(analysis_df[['Date', 'Kebele', 'Population', 'Kebele_Cumulative_Subscribers', 'Market_Penetration_Rate']].head(3).to_markdown(index=False))

Market Penetration Rate calculated and population data merged.
| Date                | Kebele   |   Population |   Kebele_Cumulative_Subscribers |   Market_Penetration_Rate |
|:--------------------|:---------|-------------:|--------------------------------:|--------------------------:|
| 2023-01-01 00:00:00 | Arada    |        21423 |                              37 |                  0.172712 |
| 2023-02-01 00:00:00 | Arada    |        21423 |                              75 |                  0.350091 |
| 2023-03-01 00:00:00 | Arada    |        21423 |                             126 |                  0.588153 |


#### 6: Calculate Month-over-Month (MoM) Growth Rate (%)

In [11]:
# 1. Get the previous month's cumulative subscribers (segmented)
analysis_df['Prev_Month_Subscribers'] = analysis_df.groupby(['Kebele', 'Segment'])['Cumulative_Subscribers'].shift(1)
analysis_df['Prev_Month_Subscribers'] = analysis_df['Prev_Month_Subscribers'].fillna(0) # First month has 0 previous subscribers

# 2. Calculate the MoM Growth Rate
analysis_df['MoM_Growth_Rate'] = (
    (analysis_df['Cumulative_Subscribers'] - analysis_df['Prev_Month_Subscribers']) / 
    analysis_df['Prev_Month_Subscribers']
) * 100

# 3. Clean up the initial NaN/Inf values (which occur in the very first month)
analysis_df['MoM_Growth_Rate'] = analysis_df['MoM_Growth_Rate'].replace([float('inf'), -float('inf')], 0).fillna(0)

# 4. Correct the MoM for the very first month (where Prev_Month_Subscribers was 0)
# A simple way for a portfolio project is to set it to 100% since all growth came from the new acquisitions
analysis_df.loc[analysis_df['Prev_Month_Subscribers'] == 0, 'MoM_Growth_Rate'] = 100.0

print("Month-over-Month Growth Rate calculated.")
print(analysis_df[['Date', 'Kebele', 'Segment', 'Cumulative_Subscribers', 'MoM_Growth_Rate']].tail(3).to_markdown(index=False))

Month-over-Month Growth Rate calculated.
| Date                | Kebele    | Segment          |   Cumulative_Subscribers |   MoM_Growth_Rate |
|:--------------------|:----------|:-----------------|-------------------------:|------------------:|
| 2024-10-01 00:00:00 | Wadu Amba | Telebirr Adopter |                      517 |           6.59794 |
| 2024-11-01 00:00:00 | Wadu Amba | Telebirr Adopter |                      550 |           6.38298 |
| 2024-12-01 00:00:00 | Wadu Amba | Telebirr Adopter |                      570 |           3.63636 |


#### 7: Final Cleanup and Export

In [12]:
analysis_df['Year'] = analysis_df['Date'].dt.year
analysis_df['Month'] = analysis_df['Date'].dt.month

final_columns = [
    'Date', 'Year', 'Month', 'Kebele', 'Type', 'Segment', 'Population',
    'New_Subscribers', 'Monthly_Churn', 'Net_Acquisition',
    'Cumulative_Subscribers', # Segment-wise
    'Kebele_Cumulative_Subscribers', # Kebele-total
    'Market_Penetration_Rate', # Kebele-total
    'MoM_Growth_Rate' # Segment-wise
]

final_analysis_df = analysis_df[final_columns].round(2)
final_analysis_df.to_csv('subscriber_analysis_data.csv', index=False)

print("\n--- Data Wrangling Complete ---")
print("New file 'subscriber_analysis_data.csv' created.")
print(f"Final data shape: {final_analysis_df.shape}")


--- Data Wrangling Complete ---
New file 'subscriber_analysis_data.csv' created.
Final data shape: (1080, 14)
